In [ ]:
import numpy as np
import pandas as pd
from warnings import filterwarnings as f_w
f_w('ignore')
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500

In [ ]:
table = 'pos'
locator = pd.read_excel('locator.xlsx', index_col = 0)

In [ ]:
pos = pd.read_csv(locator.loc[table, 'initial'])

In [ ]:
prev = pd.read_csv(locator.loc['prev', 'initial'],
                   usecols = ['SK_ID_PREV', 'AMT_ANNUITY', 
                              'CNT_PAYMENT'])

In [ ]:
pos = pd.merge(pos,
               prev,
               on = 'SK_ID_PREV',
               how = 'left')
pos['CNT_PAYMENT_FACT_RATIO'] = pos.eval('CNT_INSTALMENT / CNT_PAYMENT')
pos['FUTURE_PAYMENT_RATIO'] = pos.eval('CNT_INSTALMENT_FUTURE / CNT_PAYMENT')
pos['REMAINED_TO_PAY'] = pos.eval('CNT_INSTALMENT_FUTURE * AMT_ANNUITY')
pos = pos.drop(columns = ['CNT_PAYMENT', 'AMT_ANNUITY'])
pos = pos.sort_values(by = ['SK_ID_PREV', 'MONTHS_BALANCE'])

In [ ]:
pos_last = pos.groupby('SK_ID_PREV').agg({'SK_ID_CURR': 'max',
                                          'CNT_PAYMENT_FACT_RATIO': 'last',
                                          'FUTURE_PAYMENT_RATIO': 'last',
                                          'REMAINED_TO_PAY': 'last',
                                          'CNT_INSTALMENT_FUTURE': 'last',
                                        })

In [ ]:
pos_agg = pos_last.groupby('SK_ID_CURR').agg({'CNT_PAYMENT_FACT_RATIO': ['min', 'max', 'mean', 'std'],
                                              'FUTURE_PAYMENT_RATIO': ['min', 'max', 'mean', 'std'],
                                              'REMAINED_TO_PAY': ['sum'],
                                              'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'std'],
                                             })

In [ ]:
pos_agg = pos_agg.join(pos.groupby('SK_ID_CURR').agg({'SK_DPD': ['max', 'mean', 'std'],
                                                      'SK_DPD_DEF': ['max', 'mean', 'std'],
                                                     }))

In [ ]:
pos_agg.columns = pd.Index(['_'.join(('POS', c[0], c[1].upper())) for c in pos_agg.columns])

In [ ]:
pos_agg = pos_agg.replace({np.inf:np.nan,
                           -np.inf:np.nan})

In [ ]:
pos_agg.to_csv(locator.loc[table, 'feat_eng'])